In [12]:
from recommendations import critics
import recommendations

In [2]:
critics['Lisa Rose']['Lady in the Water']

2.5

In [3]:
critics['Toby']['Snakes on a Plane']=4.5

In [4]:
critics['Toby']

{'Snakes on a Plane': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 1.0}

In [5]:
#for example calculate the distance between two point
from math import sqrt
sqrt(pow(5-4,2)+pow(4-1,2))

3.1622776601683795

In [6]:
1/(1+sqrt(pow(5-4,2)+pow(4-1,2)))

0.2402530733520421

# Euclidean Distance Score

In [7]:
##Make a function to wrap them up
#The code for sim_distance
from math import sqrt
#Return a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    #Get the list of shared_items
    si = {}
    for item in prefs[preson1]:
        if item in prefs[person2]:
            si[item] = 1
#if they have no ratings in common, return 0
    if len(si)==0: return 0
#Add up the squares of all the differences
    sum_of_squares = sum([pow(prefs[person1][item]-prefs[person2][item],2)
                          for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)

In [17]:
reload(recommendations)

<module 'recommendations' from 'recommendations.py'>

In [18]:
recommendations.sim_distance(recommendations.critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

# Pearson Correlation Score

In [19]:
#Return the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    #Get the list of mutually rated items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:si[item]=1
    #Find the number of elements
    n = len(si)
    #If they are no ratings in common, return 0
    if n==0:return 0
    #Add yup all tje preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
    
    #Sum up the squares 
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    
    #sum up the products
    pSum = sum([prefs[p1][it]*prefs[p2][it] for it in si])
    
    #Calculate Pearson score
    num = pSum-(sum1*sum2/n)
    den = sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0:return 0
    r = num/den
    return r

In [31]:
reload(recommendations)
recommendations.sim_pearson(recommendations.critics,
                           'Lisa Rose','Gene Seymour')

0.39605901719066977

In [32]:
###Ranking the Critics

In [ ]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.

In [33]:
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other)
           for other in prefs if other!=person]
    #sort the list so the highest scores appear at the top
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [38]:
reload(recommendations)

<module 'recommendations' from 'recommendations.py'>

In [39]:
recommendations.topMatches(recommendations.critics,'Toby',n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

In [ ]:
##Recommending Items

In [40]:
#Gets recommendations for a person by using a weighted average
#of every other user;s rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        #don't compare me to myself
        if other == person:continue
        sim = similarity(prefs,person,other)
        #ignore scores of zero or lower
        if sim<=0:continue
        for item in prefs[other]:
            #only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
                #Similarity*score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                #Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
        #Creating the normalized list
        rankings=[(total/simSums[item],item) for item,total in totals.items()]
        #Return the sorted list
        rankings.sort()
        rankings.reverse()
        return rankings

In [47]:
reload(recommendations)
pass

In [48]:
recommendations.getRecommendations(recommendations.critics,'Toby')

[(3.0, 'The Night Listener'), (3.0, 'Lady in the Water')]

In [49]:
recommendations.getRecommendations(recommendations.critics,'Toby',
                similarity=recommendations.sim_distance)

[(2.9999999999999996, 'The Night Listener'),
 (2.9999999999999996, 'Lady in the Water')]

In [64]:
###Matching Products
#Swap the people to items

In [65]:
'''
{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},
    'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}
to:
 {'Lady in the Water':{'Lisa Rose':2.5,'Gene Seymour':3.0},
'Snakes on a Plane':{'Lisa Rose':3.5,'Gene Seymour':3.5}}
'''

"\n{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5},\n    'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5}}\nto:\n {'Lady in the Water':{'Lisa Rose':2.5,'Gene Seymour':3.0},\n'Snakes on a Plane':{'Lisa Rose':3.5,'Gene Seymour':3.5}}\n"

In [ ]:
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
        #Flip them and person
        result[item][person]=prefs[person][item]
    return result

In [72]:
reload(recommendations)
pass

In [73]:
movies = recommendations.transformPrefs(recommendations.critics)
recommendations.topMatches(movies,'Superman Returns')

[(0.6579516949597695, 'You, Me and Dupree'),
 (0.4879500364742689, 'Lady in the Water'),
 (0.11180339887498941, 'Snakes on a Plane'),
 (-0.1798471947990544, 'The Night Listener'),
 (-0.42289003161103106, 'Just My Luck')]

In [74]:
#To twist things around even more, you can get recommended critics for a movie.

In [75]:
recommendations.getRecommendations(movies,'Just My Luck')

[(4.0, 'Michael Phillips'), (3.0, 'Jack Matthews')]

In [76]:
##The del.icio.us API

In [1]:
import pydelicious

In [2]:
pydelicious.get_popular(tag='python')

[{'count': '',
  'description': u'something went wrong',
  'dt': '',
  'extended': '',
  'hash': '',
  'href': '',
  'tags': '',
  'user': ''}]